# Python Performance Tips
https://wiki.python.org/moin/PythonSpeed/PerformanceTips

In this lecture we will understand the impact of
- Built-in functions
- Function Call Overhead
- Function Decorator
- Loops and "."

in the performance of a code.

----
## Built-in functions
One of the easiest ways to improve Python performance is to use built-in functions! Python provides a large number of built-in functions that perform a wide variety of operations. These built-in functions are written in C, so they run quite efficiently.

In [1]:
import random

def my_min(values):
    min_value = values[0]
    for v in values:
        if v < min_value:
            min_value = v
    return min_value

k = 10000

random_numbers = [random.randint(0,100) for p in range(0,k)]

# using the magic function %timeit
%timeit -n 100 my_min(random_numbers) 

%timeit -n 100 min(random_numbers)

362 µs ± 63.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
164 µs ± 2.95 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


----
## Function Call Overhead
Function call overhead in Python is relatively high, especially compared with the execution speed of builtin functions. The overhead in Python is mainly due to the dynamic type checking of function arguments that must be performed before and after the function call. 

In [12]:
%%timeit -n 100
import time

def inner(i,x):
    x = x + i
    return(x)

x = 0
for i in range(10000): 
    x = inner(i,x)
#print(x)

1.58 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%%timeit -n 100
def inner2(l):
    x = 0
    
    for i in l:
        x = x + i
    return(x)

x = inner2(range(10000))
#print(x)

651 µs ± 77.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Function Decorator
The symbol @ is Python decorator syntax. Python decorators are callable Python object that is used to modify a function, method or class definition. The wise use of decorators can improve the performance of codes.

How does a decorator work?

In [4]:
def decorating_a_function(func):
    def function_wrapper(x):
        print("Now \"" + func.__name__ + "\" does much more than simply print its attribute")
        print("The attribute")
        func(x)
        print("is surrounded by all these text !")
    return function_wrapper

@decorating_a_function
def foo(x):
    print(str(x))

foo("Hi")

Now "foo" does much more than simply print its attribute
The attribute
Hi
is surrounded by all these text !


Third party functions can also be decorated.

In [5]:
from math import sin, cos

def our_decorator(func):
    def function_wrapper(x):
        print("This is the result of " + func.__name__ + "("+str(x)+")")
        print("%.5f" % func(x))
    return function_wrapper

# in this case is not possible to use @
sin = our_decorator(sin)
cos = our_decorator(cos)

for f in [sin, cos]:
    f(3.1415)

This is the result of sin(3.1415)
0.00009
This is the result of cos(3.1415)
-1.00000


Using _wraps_ from _functools_

In [6]:
def greeting(func):
    def function_wrapper(x):
        """ function_wrapper of greeting """
        print("Hi, the function \"" + func.__name__ + "\" returns: ", func(x))
    return function_wrapper

@greeting
def f(x):
    """ just some silly function """
    return(x + 4)

f(10)
print("------")
print("function name: " + f.__name__)
print("docstring: " + f.__doc__)
print("module name: " + f.__module__)

Hi, the function "f" returns:  14
------
function name: function_wrapper
docstring:  function_wrapper of greeting 
module name: __main__


In [7]:
from functools import wraps

def greeting(func):
    @wraps(func)
    def function_wrapper(x):
        """ function_wrapper of greeting """
        print("Hi, the function \"" + func.__name__ + "\" returns: ", func(x))
    return function_wrapper

@greeting
def f(x):
    """ just some silly function """
    return(x + 4)

f(10)
print("------")
print("function name: " + f.__name__)
print("docstring: " + f.__doc__)
print("module name: " + f.__module__)

Hi, the function "f" returns:  14
------
function name: f
docstring:  just some silly function 
module name: __main__


Using decorators for CACHING

In [8]:
def fib(i):
    if i < 2: return 1
    return fib(i-1) + fib(i-2)

t = time.process_time() 

fibo = fib(30)

elapsed_time1 = time.process_time() - t
print('time and result for fib: ',elapsed_time1, fibo)

NameError: name 'time' is not defined

In [ ]:
from functools import wraps

def cache(f):
    cache = { }
    @wraps(f)
    def function_wrapper(*arg):
        if arg not in cache: cache[arg] = f(*arg)
        return cache[arg]
    return function_wrapper

@cache
def fib(i):
    if i < 2: return 1
    return fib(i-1) + fib(i-2)

t = time.process_time() 

fibo = fib(30)

elapsed_time1 = time.process_time() - t
print('time and result for fib: ',elapsed_time1, fibo)

## Loops
They key to optimising loops is to minimize what they do.<br>
Lets see the effect of a "." operator within a loop.

In [15]:
import random

lowerlist = ['abcdefghijklmnopqrstuvwxyz'[:random.randint(0,25)] for x in range(1000)]
upperlist = []

def to_upper_1(lowerlist):
    for word in lowerlist:
        upperlist.append(str.upper(word))  # here is the 
  
%timeit -n 10 to_upper_1(lowerlist)
#print(upperlist[:10])

606 µs ± 57.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit -n 10 

upperlist = []
upper = str.upper          # this create references to the methods,
append = upperlist.append  # avoiding searches

def to_upper_2(lowerlist):
    for word in lowerlist:
        append(upper(word))
        
to_upper_2(lowerlist)

343 µs ± 19.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%timeit -n 10
# avoiding the loop altogether
upper = str.upper
upperlist = list(map(upper, lowerlist))

229 µs ± 46.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit -n 10
upper = str.upper 
upperlist = [upper(x) for x in lowerlist]

168 µs ± 14 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
